In [2]:
import numpy as np
import pandas as pd


In [3]:
jan23 = pd.read_csv('payroll/Payroll_Report_-_End_Date_1_23_2019.csv')


In [4]:
jan23.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1248 entries, 0 to 1247
Data columns (total 11 columns):
EMPLOYEE_NAME            1248 non-null object
OFFICE                   1248 non-null object
CITY                     1248 non-null object
EMPLOYEE_TITLE           1248 non-null object
BIWEEKLY/HOURLY_RATE     1248 non-null float64
PAYROLL_TYPE             1248 non-null object
PAY_PERIOD               1248 non-null int64
PAY_PERIOD_BEGIN_DATE    1248 non-null object
PAY_PERIOD_END_DATE      1248 non-null object
CHECK_DATE               1248 non-null object
LEGISLATIVE_ENTITY       1248 non-null object
dtypes: float64(1), int64(1), object(9)
memory usage: 107.3+ KB


In [6]:
jan23.head(3)


,EMPLOYEE_NAME,OFFICE,CITY,EMPLOYEE_TITLE,BIWEEKLY/HOURLY_RATE,PAYROLL_TYPE,PAY_PERIOD,PAY_PERIOD_BEGIN_DATE,PAY_PERIOD_END_DATE,CHECK_DATE,LEGISLATIVE_ENTITY
0,"ABBOT, JUDITH L",SENATOR TOBY ANN STAVISKY,FLUSHING,COMMUNITY LIAISON,1076.93,SA,21,01/10/2019 12:00:00 AM,01/23/2019 12:00:00 AM,02/06/2019 12:00:00 AM,SENATE EMPLOYEE
1,"ABRAHAM, PRINCY A",MINORITY COUNSEL/PROGRAM,ALBANY,ASSOCIATE COUNSEL,2376.93,RA,21,01/10/2019 12:00:00 AM,01/23/2019 12:00:00 AM,02/06/2019 12:00:00 AM,SENATE EMPLOYEE
2,"ABREU, BRIAN A",LEGISLATIVE MESSENGER SERVICE,ALBANY,MESSENGER,1173.93,RA,21,01/10/2019 12:00:00 AM,01/23/2019 12:00:00 AM,02/06/2019 12:00:00 AM,JOINT LEGISLATIVE EMPLOYEE


In [5]:
columns = ['EMPLOYEE_NAME', 'OFFICE', 'CITY', 'EMPLOYEE_TITLE', 'BIWEEKLY/HOURLY_RATE', 'PAYROLL_TYPE', 'LEGISLATIVE_ENTITY']
jan_mod = jan23[columns].sort_values(by = ['OFFICE'])
jan_mod.tail(25)

,EMPLOYEE_NAME,OFFICE,CITY,EMPLOYEE_TITLE,BIWEEKLY/HOURLY_RATE,PAYROLL_TYPE,LEGISLATIVE_ENTITY
1119,"STOUFFER, SAM L",STS/BUSINESS APPLICATIONS CENTER,ALBANY,SENIOR WEB DEVELOPER,2786.26,RA,SENATE EMPLOYEE
513,"HEAPHY, ALISON G",STS/BUSINESS APPLICATIONS CENTER,ALBANY,DRUPAL ARCHITECT,3345.92,RA,SENATE EMPLOYEE
123,"BOUCHER, JOHN C",STS/BUSINESS APPLICATIONS CENTER,ALBANY,PROGRAMMER,3481.95,RA,SENATE EMPLOYEE
105,"BINKOWSKI, STEVEN B",STS/BUSINESS APPLICATIONS CENTER,ALBANY,WEB DEVELOPER,2611.75,RA,SENATE EMPLOYEE
463,"GORDINIER, CHRISTINE M",STS/BUSINESS APPLICATIONS CENTER,ALBANY,"SUPERVISOR, APPLICATIONS PROGRAMMING",3416.12,RA,SENATE EMPLOYEE
427,"GARDNER, STEPHEN J",STS/BUSINESS APPLICATIONS CENTER,ALBANY,DATABASE ADMINISTRATOR,73.50,TE,SENATE EMPLOYEE
615,"KING, KIMBERLY S",STS/BUSINESS APPLICATIONS CENTER,ALBANY,WEB SUPPORT SPECIALIST,2190.63,RA,SENATE EMPLOYEE
1018,"SAINTIL, FANES K",STS/BUSINESS APPLICATIONS CENTER,ALBANY,WEB SUPPORT SPECIALIST,2634.14,RA,SENATE EMPLOYEE
517,"HEITNER, BRIAN",STS/BUSINESS APPLICATIONS CENTER,ALBANY,SENIOR PROGRAMMER,3469.05,RA,SENATE EMPLOYEE
1090,"SNYDER, SETH M",STS/BUSINESS APPLICATIONS CENTER,HOLLYWOOD,SENIOR PROGRAMMER,3672.35,RA,SENATE EMPLOYEE


In [12]:
def isOneToOne(df, col1, col2):
    first = df.drop_duplicates([col1, col2]).groupby(col1)[col2].count().max()
    second = df.drop_duplicates([col1, col2]).groupby(col2)[col1].count().max()
    return first + second == 2

isOneToOne(jan_mod, 'LEGISLATIVE_ENTITY', 'EMPLOYEE_NAME')

first = jan_mod.groupby('LEGISLATIVE_ENTITY')['OFFICE'].count().max()
second = jan_mod.groupby('OFFICE')['LEGISLATIVE_ENTITY'].count().max()
print(first, second)

1205 63
